In [12]:
import os

from kompy import KomootConnector
import os

In [13]:
connector = KomootConnector(
    email=os.environ['KOMOOT_EMAIL'],
    password=os.environ['KOMOOT_PASSWORD'],
)

In [14]:
tours = connector.get_tours()

2023-11-17 14:11:44,626 - KomootConnector - WARNING - No user identifier provided, using the currently logged user: 2453610435162
2023-11-17 14:11:44,626 No user identifier provided, using the currently logged user: 2453610435162
2023-11-17 14:11:44,638 - KomootConnector - WARNING - No sort field provided, using default sort field: date
2023-11-17 14:11:44,638 No sort field provided, using default sort field: date
2023-11-17 14:11:48,595 No vector map image found.
2023-11-17 14:11:48,599 No vector map image found.
2023-11-17 14:11:48,599 No vector map image found.
2023-11-17 14:11:48,600 No vector map image found.
2023-11-17 14:11:48,601 No vector map image found.
2023-11-17 14:11:48,602 No vector map image found.
2023-11-17 14:11:48,604 No vector map image found.
2023-11-17 14:11:48,604 No vector map image found.
2023-11-17 14:11:48,605 No vector map image found.
2023-11-17 14:11:48,605 No vector map image found.
2023-11-17 14:11:48,606 No vector map image found.
2023-11-17 14:11:48,6

In [15]:
tour1 = tours[0]

In [16]:
tour1.generate_gpx_track(authentication=connector.authentication)

In [17]:
gpx_track = tour1.gpx_track

In [18]:
for track in gpx_track.tracks:
    print(track)
    for segment in track.segments:
        print(segment)
        for point in segment.points:
            print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevation}')

GPXTrack(name='W01D6-Fartlek Run', segments=[GPXTrackSegment(points=[...])])
GPXTrackSegment(points=[...])
Point at (44.798193,7.201098) -> 1190.560541
Point at (44.798197,7.201093) -> 1190.560541
Point at (44.7982,7.201092) -> 1190.560541
Point at (44.798203,7.201094) -> 1190.560541
Point at (44.798212,7.201095) -> 1190.560541
Point at (44.798223,7.201094) -> 1190.560541
Point at (44.798237,7.201092) -> 1190.560541
Point at (44.798254,7.201092) -> 1190.560541
Point at (44.798269,7.201091) -> 1190.560541
Point at (44.798292,7.201088) -> 1190.560541
Point at (44.798315,7.201082) -> 1190.560541
Point at (44.798343,7.201078) -> 1190.560541
Point at (44.798367,7.201072) -> 1190.560541
Point at (44.798396,7.201063) -> 1190.560541
Point at (44.798422,7.201055) -> 1190.560541
Point at (44.798447,7.201039) -> 1190.560541
Point at (44.798476,7.201027) -> 1190.560541
Point at (44.798492,7.201015) -> 1190.560541
Point at (44.798515,7.200998) -> 1190.560541
Point at (44.79853,7.200981) -> 1190.560

In [19]:
for waypoint in gpx_track.waypoints:
    print(f'waypoint {waypoint.name} -> ({waypoint.latitude},{waypoint.longitude})')

In [20]:
for route in gpx_track.routes:
    print('Route:')
    for point in route.points:
        print(f'Point at ({point.latitude},{point.longitude}) -> {point.elevtion}')


In [85]:
from gpxpy.gpx import GPXTrackPoint
from geopy import distance
import math

def calculate_initial_compass_bearing(first_point: GPXTrackPoint, second_point: GPXTrackPoint):
    """
    Calculates the bearing between two points.

    The formula used to calculate the bearing is:
        θ = atan2(sin(Δlong).cos(lat2), cos(lat1).sin(lat2) - sin(lat1).cos(lat2).cos(Δlong))

    :param first_point: The first point
    :param second_point: The second point
    :return: The bearing between the two points in degrees
    """

    lat1 = math.radians(first_point.latitude)
    lat2 = math.radians(second_point.latitude)
    diffLong = math.radians(second_point.longitude - first_point.longitude)

    x = math.sin(diffLong) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(diffLong))

    initial_bearing = math.atan2(x, y)

    # Normalize the initial bearing
    initial_bearing = math.degrees(initial_bearing)
    compass_bearing = (initial_bearing + 360) % 360

    return compass_bearing



def process_gpx_for_turns(gpx_data):
    processed_segments = []
    for track in gpx_data.tracks:
        for segment in track.segments:
            # Initialize previous bearing
            previous_bearing = None

            for i in range(len(segment.points)-1):
                point1 = segment.points[i]
                point2 = segment.points[i+1]

                # Calculate the elevation change
                elevation_change = point2.elevation - point1.elevation
                m_distance = distance.geodesic(
                        (point1.latitude, point1.longitude),
                        (point2.latitude,point2.longitude)
                    ).m
                slope_angle = math.atan(elevation_change/m_distance)
                segment_type = "Uphill" if elevation_change > 0 else "Downhill" if elevation_change < 0 else "Flat"

                # Calculate the bearing
                if previous_bearing is not None:
                    current_bearing = calculate_initial_compass_bearing(
                        first_point=point1,
                        second_point=point2,
                    )
                    bearing_change = current_bearing - previous_bearing

                    # Determine left or right turn
                    if bearing_change > 10:
                        turn_direction = "Right"
                    elif bearing_change < -10:
                        turn_direction = "Left"
                    else:
                        turn_direction = "Straight"

                    # Classify the turn strength
                    if turn_direction != "Straight":
                        if abs(bearing_change) > 45:
                            turn_strength = "Strong"
                        else:
                            turn_strength = "Mild"
                    else:
                        turn_strength = "N/A"

                    previous_bearing = current_bearing
                else:
                    turn_direction = "N/A"
                    turn_strength = "N/A"
                    previous_bearing = calculate_initial_compass_bearing(
                        first_point=point1,
                        second_point=point2,
                    )
                processed_segments.append({
                    'segment': i,
                    'latitude': point1.latitude,
                    'longitude': point1.longitude,
                    'altitude': point1.elevation,
                    'elevation_change': elevation_change,
                    'segment_type': segment_type,
                    'turn_direction': turn_direction,
                    'turn_strength': turn_strength,
                    'distance': m_distance,
                    'slope_angle': slope_angle,
                })

    return processed_segments

In [86]:
processed_segments = process_gpx_for_turns(gpx_track)

In [87]:
import pandas as pd
df = pd.DataFrame(processed_segments)
df['cumulative_distance'] = df.distance.cumsum()
df['cumulative_elevation'] = df.elevation_change.cumsum()
print(df.shape)
display(df[1100:2000])

(2505, 12)


,segment,latitude,longitude,altitude,elevation_change,segment_type,turn_direction,turn_strength,distance,slope_angle,cumulative_distance,cumulative_elevation
1100,1100,44.791758,7.179772,1317.823446,0.179864,Uphill,Left,Mild,2.359223,0.076091,2648.113651,127.442769
1101,1101,44.791738,7.179762,1318.003310,0.208327,Uphill,Right,Mild,2.734029,0.076051,2650.847680,127.651096
1102,1102,44.791717,7.179744,1318.211637,0.192024,Uphill,Straight,N/A,2.519656,0.076063,2653.367337,127.843120
1103,1103,44.791697,7.179729,1318.403661,0.158548,Uphill,Straight,N/A,2.080061,0.076076,2655.447398,128.001668
1104,1104,44.791680,7.179718,1318.562209,0.184586,Uphill,Straight,N/A,2.422198,0.076059,2657.869596,128.186254
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,44.794531,7.192561,1181.237729,-0.308019,Downhill,Straight,N/A,2.944545,-0.104228,4943.870734,-9.630831
1996,1996,44.794540,7.192596,1180.929710,-0.306805,Downhill,Straight,N/A,2.932033,-0.104260,4946.802767,-9.937636
1997,1997,44.794552,7.192629,1180.622905,-0.321621,Downhill,Straight,N/A,3.073816,-0.104253,4949.876584,-10.259257
1998,1998,44.794564,7.192664,1180.301284,-0.329091,Downhill,Straight,N/A,3.145297,-0.104250,4953.021881,-10.588348


In [88]:
tour1.generate_coordinates(authentication=connector.authentication)

True

In [89]:
tour1.constitution.

SyntaxError: invalid syntax (4200940938.py, line 1)